***GENERATED CODE FOR dataprepapp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run dataprepappHooks.ipynb
try:
	#sourcePreExecutionHook()

	bostontrain = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/regression/BostonTrain.csv', 'filename': 'BostonTrain.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(bostontrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dataprepappHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bostontrain,spark,json.dumps( {"url": "/FileStore/platform/Data/TrainData/regression/BostonTrain.csv", "data_source": "hdfs", "BasicStats": {"missingValues": 0, "numberOfColumns": 14, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "type": "real", "max": "89.0", "min": "0.0", "mean": "8.6", "missing": 0, "stddev": 8.60154510533249}, {"column": "ZN", "alias": "ZN", "type": "real", "max": "100.0", "min": "0.0", "mean": "23.3", "missing": 0, "stddev": 23.32245299451514, "changeColumn": 1}, {"column": "INDUS", "alias": "INDUS", "type": "real", "max": "27.7", "min": "0.5", "mean": "6.9", "missing": 0, "stddev": 6.860352940897585}, {"column": "CHAS", "alias": "CHAS", "type": "real", "max": "1.0", "min": "0.0", "mean": "0.3", "missing": 0, "stddev": 0.25399404134041037}, {"column": "NOX", "alias": "NOX", "type": "real", "max": "0.9", "min": "0.4", "mean": "0.1", "missing": 0, "stddev": 0.11587767566755595}, {"column": "RM", "alias": "RM", "type": "real", "max": "8.8", "min": "3.6", "mean": "0.7", "missing": 0, "stddev": 0.7026171434153233}, {"column": "AGE", "alias": "AGE", "type": "real", "max": "100.0", "min": "2.9", "mean": "28.1", "missing": 0, "stddev": 28.148861406903617}, {"column": "DIS", "alias": "DIS", "type": "real", "max": "12.1", "min": "1.1", "mean": "2.1", "missing": 0, "stddev": 2.105710126627611}, {"column": "RAD", "alias": "RAD", "type": "real", "max": "24.0", "min": "1.0", "mean": "8.7", "missing": 0, "stddev": 8.707259384239366}, {"column": "TAX", "alias": "TAX", "type": "real", "max": "711.0", "min": "187.0", "mean": "168.5", "missing": 0, "stddev": 168.53711605495903}, {"column": "PTRATIO", "alias": "PTRATIO", "type": "real", "max": "22.0", "min": "12.6", "mean": "2.2", "missing": 0, "stddev": 2.1649455237144406}, {"column": "B", "alias": "B", "type": "real", "max": "396.9", "min": "0.3", "mean": "91.3", "missing": 0, "stddev": 91.29486438415783}, {"column": "LSTAT", "alias": "LSTAT", "type": "real", "max": "38.0", "min": "1.7", "mean": "7.1", "missing": 0, "stddev": 7.141061511348571}, {"column": "MEDV", "alias": "MEDV", "type": "real", "max": "50.0", "min": "5.0", "mean": "9.2", "missing": 0, "stddev": 9.197104087379818}]}, "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/5f8d34163aacdd4e73ca0bd91603091829/0part.csv", "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "ZN", "type": "real", "min": 0, "max": 100, "mean": 11.4}], "functionsData": [{"replaceType": "none", "toReplace": "", "replaceValueType": "max", "ReplaceWith": ""}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "position": 11, "type": "real"}, {"field": "B", "alias": "B", "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "position": 14, "type": "real"}], "requestRatio": 1, "totalRows": 506, "functionChanges": [{"columnName": "ZN", "functionName": "Replace Outliers", "Type": "real", "Parameters": [{"replaceType": "none", "toReplace": "", "replaceValueType": "max", "ReplaceWith": ""}]}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataprepappHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "4.52", "stddev": "13.45", "min": "0.01301", "max": "88.9762", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "8.86", "stddev": "18.55", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "10.47", "stddev": "6.03", "min": "1.32", "max": "19.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "0.04", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "0.55", "stddev": "0.12", "min": "0.405", "max": "0.871", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "6.36", "stddev": "0.77", "min": "4.926", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "69.83", "stddev": "26.4", "min": "6.5", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "3.94", "stddev": "2.16", "min": "1.4165", "max": "8.9067", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "8.98", "stddev": "8.11", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "397.92", "stddev": "152.65", "min": "193.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "18.09", "stddev": "2.06", "min": "14.7", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "351.22", "stddev": "102.24", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "12.4", "stddev": "6.99", "min": "2.98", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "51", "mean": "21.9", "stddev": "9.79", "min": "7.4", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run dataprepappHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

